In [ ]:
#open file


In [ ]:
#create list of dictionaries

In [ ]:
#read lines from file, splitting by '; ' and storing multiple entries

In [ ]:
#transform into flat file with internal ID, GUID, and 

In [ ]:
#load GUID file

In [ ]:
#problem to solve for looking up GUID: when strand/substrand gives context to component, 
#if semi-colon separated, then will have to allow for non-matches